<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Анализ" data-toc-modified-id="Анализ-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Анализ</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LinearRegression" data-toc-modified-id="LinearRegression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>LinearRegression</a></span></li><li><span><a href="#RandomForestRegressor" data-toc-modified-id="RandomForestRegressor-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>RandomForestRegressor</a></span></li><li><span><a href="#CatBoostRegressor" data-toc-modified-id="CatBoostRegressor-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>CatBoostRegressor</a></span></li></ul></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование</a></span></li></ul></div>

#  Прогнозирование заказов такси

Компания «Чётенькое такси» собрала исторические данные о заказах такси в аэропортах. Чтобы привлекать больше водителей в период пиковой нагрузки, нужно спрогнозировать количество заказов такси на следующий час. Постройте модель для такого предсказания.

Значение метрики *RMSE* на тестовой выборке должно быть не больше 48.

Вам нужно:

1. Загрузить данные и выполнить их ресемплирование по одному часу.
2. Проанализировать данные.
3. Обучить разные модели с различными гиперпараметрами. Сделать тестовую выборку размером 10% от исходных данных.
4. Проверить данные на тестовой выборке и сделать выводы.


Данные лежат в файле `taxi.csv`. Количество заказов находится в столбце `num_orders` (от англ. *number of orders*, «число заказов»).

## Подготовка

Загрузим библиотеки

In [60]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, make_scorer

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from statsmodels.tsa.seasonal import seasonal_decompose

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [12]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Откроем файл

In [13]:
df_1 = pd.read_csv('datasets/taxi.csv', index_col=[0], parse_dates=[0])

In [14]:
df_1.head()

,num_orders
datetime,
2018-03-01 00:00:00,9
2018-03-01 00:10:00,14
2018-03-01 00:20:00,28
2018-03-01 00:30:00,20
2018-03-01 00:40:00,32


Подготовим данные

In [19]:
data = df_1.resample('1H').sum()
print(data.head())
print(data.tail())
print('')
print('Начало периода ', data.index.min(), 'конец периода ', data.index.max())
print('Продолжительность ', data.index.max() - data.index.min())

                     num_orders
datetime                       
2018-03-01 00:00:00         124
2018-03-01 01:00:00          85
2018-03-01 02:00:00          71
2018-03-01 03:00:00          66
2018-03-01 04:00:00          43
                     num_orders
datetime                       
2018-08-31 19:00:00         136
2018-08-31 20:00:00         154
2018-08-31 21:00:00         159
2018-08-31 22:00:00         223
2018-08-31 23:00:00         205

Начало периода  2018-03-01 00:00:00 конец периода  2018-08-31 23:00:00
Продолжительность  183 days 23:00:00


## Анализ

Посмотрим на статистические характеристики

In [20]:
data.describe()

,num_orders
count,4416.000000
mean,84.422781
std,45.023853
min,0.000000
25%,54.000000
50%,78.000000
75%,107.000000
max,462.000000


Рассмотрим трендовые и сезонные составляющие 

In [140]:
fig = make_subplots(
    rows=3, cols=1,
    row_heights=[1, 1, 1],
    subplot_titles=('Trend',  'Seasonal', 'Residuals'),
    x_title=('time'),
    y_title=('orders')
    
)


fig.add_trace(go.Scatter(y = decomposed.trend,
                    mode='lines',    
                    name="Trend"), row=1, col=1)
fig.add_trace(go.Scatter(y=decomposed.seasonal[:96],
                    mode='lines+markers',
                    name='Seasonal'), row=2, col=1)
fig.add_trace(go.Scatter(y=decomposed.resid,
                    mode='lines', 
                    name='Residuals'), row=3, col=1)


fig.update_layout(
    autosize=False,
    width=900,
    height=700,
    title_text="Decompose of the time series",
    title_font_size=20
)

fig.show()

Вывод: видим выраженый тренд, заказов становится больше. Сезонность также прослеживается, каждые 24 часа.

## Обучение

Разделим датасет на обучающую и тестовую выборки

In [141]:
train, test = train_test_split(data, shuffle=False, test_size=0.1)

print(train.index.min(), train.index.max())
print(test.index.min(), test.index.max())

2018-03-01 00:00:00 2018-08-13 13:00:00
2018-08-13 14:00:00 2018-08-31 23:00:00


In [142]:
print("Среднее количество заказов в час:", test['num_orders'].mean())

pred_median = np.ones(test.shape) * train['num_orders'].median()

print("RMSE:", mean_squared_error(test, pred_median) ** 0.5)

Среднее количество заказов в час: 139.55656108597285
RMSE: 87.15277582981295


Оценим модель предыдущим значением ряда

In [143]:
pred_previous = np.ones(test.shape) * test.shift().fillna(train.iloc[len(train)-1])
print("RMSE:", mean_squared_error(test, pred_previous) ** 0.5)

RMSE: 58.856486242815066


При таком прогнозе модель ошибается меньше. Проверка на адекватность есть.

Добавим функцию для описания признаков

In [144]:
def make_features(df, max_lag, rolling_mean_size):    
    df_new = df.copy(deep=True)
    df_new['year'] = df_new.index.year
    df_new['month'] = df_new.index.month
    df_new['day'] = df_new.index.day
    df_new['dayofweek'] = df_new.index.dayofweek
    
    for lag in range(1, max_lag + 1):
        df_new['lag_{}'.format(lag)] = df_new['num_orders'].shift(lag)

    df_new['rolling_mean'] = df_new['num_orders'].shift().rolling(rolling_mean_size).mean()
    return df_new

Обучим модель на полученных признаках

In [147]:
data_feature = make_features(data, 48, 36)

train, test = train_test_split(data_feature, shuffle=False, test_size=0.1)
train = train.dropna()

features_train = train.drop('num_orders', axis = 1)
target_train = train['num_orders']

features_test = test.drop('num_orders', axis = 1)
target_test = test['num_orders']


Добавим метрику

In [148]:
mse = make_scorer(mean_squared_error, greater_is_better=False)

###  LinearRegression

Подберем гиперпараметры модели с помощью GridSearchCV

In [162]:
model = LinearRegression()
parameters = {'fit_intercept':[True,False], 
              'normalize':[True,False], 
              'copy_X':[True, False]
             }
grid = GridSearchCV(model,parameters, cv=TimeSeriesSplit(3), scoring=mse)
grid.fit(features_train, target_train)
print("best score: ", (-grid.best_score_)** 0.5, "best parameters", grid.best_params_)

best score:  25.38785791081264 best parameters {'copy_X': True, 'fit_intercept': False, 'normalize': True}


In [163]:
model_lr = grid.best_estimator_
model_lr.fit(features_train, target_train)
pred_train = model_lr.predict(features_train)

print("RMSE обучающей выборки:", mean_squared_error(target_train, pred_train)** 0.5)


RMSE обучающей выборки: 23.347323316371693


### RandomForestRegressor

Подберем гиперпараметры модели с помощью GridSearchCV

In [160]:
%%time
1 + 1
model = RandomForestRegressor()
parameters = {'n_estimators':[1,60],
              'max_depth':[1,5]
             }
grid = GridSearchCV(model,parameters, cv=TimeSeriesSplit(5), scoring = mse)
grid.fit(features_train, target_train)
print("best score: ", (grid.best_score_)** 0.5, "best parameters", grid.best_params_)

best score:  nan best parameters {'max_depth': 5, 'n_estimators': 60}
CPU times: user 12.8 s, sys: 231 ms, total: 13 s
Wall time: 15.8 s


<timed exec>:8: RuntimeWarning:

invalid value encountered in double_scalars



In [161]:
model_dfr = grid.best_estimator_
model_dfr.fit(features_train, target_train)
pred_train = model_dfr.predict(features_train)

print("RMSE обучающей выборки:", mean_squared_error(target_train, pred_train)** 0.5)


RMSE обучающей выборки: 22.31597434273287


### CatBoostRegressor

Подберем гиперпараметры модели с помощью GridSearchCV

In [164]:
    model = CatBoostRegressor()
    parameters = {'depth'         : [6, 8],
                  'learning_rate' : [0.01, 0.5, 0.1, 1],
                  'iterations'    : [30, 50, 100]
                 }
    grid = GridSearchCV(estimator=model, param_grid = parameters, cv = TimeSeriesSplit(5), n_jobs=-1)
    grid.fit(features_train, target_train)    

    # Results from Grid Search
    print("\n========================================================")
    print(" Results from Grid Search " )
    print("========================================================")    
    
    print("\n The best estimator across ALL searched params:\n",
          grid.best_estimator_)
    
    print("\n The best score across ALL searched params:\n",
          grid.best_score_)
    
    print("\n The best parameters across ALL searched params:\n",
          grid.best_params_)
    
    print("\n ========================================================")

0:	learn: 36.7934623	total: 67.3ms	remaining: 6.66s
1:	learn: 35.2501306	total: 81.9ms	remaining: 4.01s
2:	learn: 33.7902741	total: 98.7ms	remaining: 3.19s
3:	learn: 32.4814083	total: 118ms	remaining: 2.83s
4:	learn: 31.3617581	total: 129ms	remaining: 2.44s
5:	learn: 30.3490452	total: 138ms	remaining: 2.17s
6:	learn: 29.5380359	total: 149ms	remaining: 1.98s
7:	learn: 28.7621897	total: 191ms	remaining: 2.2s
8:	learn: 28.0937413	total: 205ms	remaining: 2.07s
9:	learn: 27.4814534	total: 227ms	remaining: 2.04s
10:	learn: 26.9778684	total: 246ms	remaining: 1.99s
11:	learn: 26.5374281	total: 262ms	remaining: 1.92s
12:	learn: 26.1247462	total: 379ms	remaining: 2.54s
13:	learn: 25.8061275	total: 416ms	remaining: 2.55s
14:	learn: 25.4361860	total: 453ms	remaining: 2.57s
15:	learn: 25.1230254	total: 494ms	remaining: 2.59s
16:	learn: 24.8416326	total: 532ms	remaining: 2.6s
17:	learn: 24.6276255	total: 544ms	remaining: 2.48s
18:	learn: 24.4290419	total: 562ms	remaining: 2.4s
19:	learn: 24.2518980	

In [165]:
model_cb = grid.best_estimator_
model_cb.fit(features_train, target_train)
pred_train = model_cb.predict(features_train)

print("RMSE обучающей выборки:", mean_squared_error(target_train, pred_train)** 0.5)

0:	learn: 36.7934623	total: 15.6ms	remaining: 1.55s
1:	learn: 35.2501306	total: 31ms	remaining: 1.52s
2:	learn: 33.7902741	total: 47ms	remaining: 1.52s
3:	learn: 32.4814083	total: 65.2ms	remaining: 1.56s
4:	learn: 31.3617581	total: 76.8ms	remaining: 1.46s
5:	learn: 30.3490452	total: 89.4ms	remaining: 1.4s
6:	learn: 29.5380359	total: 101ms	remaining: 1.34s
7:	learn: 28.7621897	total: 110ms	remaining: 1.27s
8:	learn: 28.0937413	total: 122ms	remaining: 1.23s
9:	learn: 27.4814534	total: 133ms	remaining: 1.19s
10:	learn: 26.9778684	total: 144ms	remaining: 1.16s
11:	learn: 26.5374281	total: 154ms	remaining: 1.13s
12:	learn: 26.1247462	total: 167ms	remaining: 1.12s
13:	learn: 25.8061275	total: 184ms	remaining: 1.13s
14:	learn: 25.4361860	total: 200ms	remaining: 1.13s
15:	learn: 25.1230254	total: 211ms	remaining: 1.11s
16:	learn: 24.8416326	total: 234ms	remaining: 1.14s
17:	learn: 24.6276255	total: 253ms	remaining: 1.15s
18:	learn: 24.4290419	total: 280ms	remaining: 1.19s
19:	learn: 24.2518980

    Лучший результат показала модель CatBoost

## Тестирование

In [252]:
pred_test_lr = model_lr.predict(features_test)
print("RMSE тестовой выборки: ", mean_squared_error(target_test, pred_test_lr)** 0.5)

RMSE тестовой выборки:  43.156705637228086


In [253]:
pred_test_dfr = model_dfr.predict(features_test)
print("RMSE тестовой выборки: ", mean_squared_error(target_test, pred_test_dfr)** 0.5)

RMSE тестовой выборки:  45.339532359795335


In [254]:
pred_test_cb = model_cb.predict(features_test)
print("RMSE тестовой выборки: ", mean_squared_error(target_test, pred_test_cb)** 0.5)

RMSE тестовой выборки:  41.38528637621573


На тестовых данных модель catboost также показала лучший результат

In [256]:
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Preds',  'Test'),
    x_title=('datetime'),
    y_title=('orders')
    
)

fig.add_trace(go.Scatter(y = pred_test_cb,
                    mode='lines',    
                    name="preds"), row=1, col=1)
fig.add_trace(go.Scatter(y=target_test,
                    mode='lines',
                    name='test'), row=2, col=1)

fig.update_layout(
    autosize=False,
    width=900,
    height=700,
    title_text="Compare predictions Catboost model and test data",
    title_font_size=20
)

fig.show()

Визуализируем предсказания моделей CB и DFR в сравнении с тестовыми данными

In [267]:
df_compare = pd.DataFrame(target_test.reset_index())
df_compare.columns = ['datetime','num_orders_test']
df_compare['num_orders_preds_cb'] = pd.Series(pred_test_cb)
df_compare['num_orders_preds_dfr'] = pd.Series(pred_test_dfr)
df_compare.index = df_compare.datetime
df_compare = df_compare.drop('datetime', axis = 1)

In [269]:
df_compare.head()

,num_orders_test,num_orders_preds_cb,num_orders_preds_dfr
datetime,,,
2018-08-13 14:00:00,102,98.793605,93.711560
2018-08-13 15:00:00,175,136.541801,130.636552
2018-08-13 16:00:00,144,163.625348,141.460970
2018-08-13 17:00:00,152,124.789052,129.924019
2018-08-13 18:00:00,104,89.834212,83.066046


Отобразим результаты на одном графике

In [266]:
fig = px.line(df_compare, y=df_compare.columns,
              title='Comparing test and preds orders')

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=12, label="12h", step="hour", stepmode="todate"),
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=7, label="7d", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(step="all")
        ])
    )
)

fig.show()

Вывод: загрузили, подготовили и проанализировали данные. Обучили модель, подобрав гиперпараметры.  Качество моделей проверено. Значение RMSE на тестовой выборке не больше 48, что удовлетворяет требованиям заказчика.